# Elixirの概要

ElixirとErlangは切っても切れない関係にある。

- ElixirはErlang VM上で動いている
  - Javaに対するKotlinやScalaと同じような位置づけ
- ElixirとErlangは相互に資産を利用し合える（Erlangの持つ長い実績を持つライブラリを使えるのはかなり強力）
- 上記のような理由から，Erlangの持つ言語的特性はElixirにも当てはまる


## Erlang

- 誕生は1986年（34年前）でかなり歴史がある
- 元々エリクソン社が電話交換機を作るために開発したため，以下のような用途に優れた特性を持っている
  - とにかくサービスを継続することが大事（耐障害性
  - アップデート時でも止められない（ホットスワップ）
  - 多数の機器で同時に利用される（分散性）
  
上記のような特徴は, 現代の分散システムでも必要とされていて，言語としてもサーバーシステムに適している。

Nintendoのゲームサーバーでも採用実績があるらしい([参考](https://employment.en-japan.com/engineerhub/entry/2019/08/01/103000))。

この特性を実現するために以下のような機能を持っている。

- 軽量プロセス
  - OSの提供するプロセスやスレッドとは異なる
  - Erlang VMによって管理されている
  - 生成オーバーヘッドがかなり小さく，大量のプロセスを性能低下無しに生成できる
    - イメージとしてはオブジェクト指向言語でクラスをインスタンス化するような気軽さで立ち上げられる
- 基本的にエラーをキャッチしない
  - エラー発生時にはプロセスを殺して，新しくプロセスを立ち上げ直す
  - つまり死活監視をするプロセスが存在して・・・）
  
ここまで聞くと欠陥がないように見えるが, もちろんそんな事は無い。特に古典的な言語のため生産性・可読性が・・・

## Elixir

書きやすく読みやすい言語でErlangの特性を利用するために開発されている。

Elixirというプログラミング言語はErlangの特性を引き継いでおり,

> Elixir is a dynamic, functional language designed for building scalable and maintainable applications.

- 動的型付け
- 関数型
- Scalable
- Maintainable
- José Valimが創始者。Plataformatec 社が中心的にサポートしている

Elixir独自の特徴（記法的な面から）としては

- パターンマッチが強力（大事．慣れると他言語に戻れないぐらい大事）
- コードの自動生成や言語拡張などができるマクロ（言語自体がマクロでどんどん拡張して発展させたもの）
  - むやみに濫用すると危険．黒魔術

## Elixirのコードの雰囲気

ここでは関数型であることと, パターンマッチの活用の部分だけ。

- 処理を関心ごとに関数に分割
- 宣言的に記述
- 小さな関数を組合せて大きな仕事を果たす
- ループや分岐, 例外処理といった制御構造を極力排除
  - 繰り返しは写像や再起関数で表す
  - 分岐や例外処理はパターンマッチで表現
  
Erlangプロセスの利用については別の機会に。
  
### 例題 FizzBuzz関数

$FizzBuzz$関数は正の整数$n$に対し, 次のように定義される。

$$
\begin{equation}
FizzBuzz(n) =
\begin{cases}
"FizzBuzz"  & \text{if $n\equiv 0\pmod {15}$,}\\
"Buzz"  & \text{if $n\equiv 0\pmod {5}$,}\\
"Fizz"  & \text{if $n\equiv 0\pmod {3}$,}\\
n  & \text{$otherwise$}
\end{cases}
\end{equation}
$$

$FizzBuzz$関数を実装し, 30以下の正の整数それぞれに対して適用した結果を表示せよ。

In [ ]:
defmodule FizzBuzz do
  @moduledoc """
  Implementation of FizzBuzz function.
  Use `FizzBuzz.say/1` to get answer of the game.
  
  Example:
  ```elixir
  iex> FizzBuzz.say(1)
  {:ok, 1}
  iex> FizzBuzz.say(3)
  {:ok, "Fizz"}
  iex> FizzBuzz.say(5)
  {:ok, "Buzz"}
  iex> FizzBuzz.say(15)
  {:ok, "FizzBuzz"}
  iex> FizzBuzz.say(0)
  {:error, "not a positive integer"}
  ```
  """
  
  @spec say(any) :: {:ok, String.t | integer} | {:error, String.t}
  def say(n) when is_integer(n) and n > 0, do: {:ok,    do_say(n)}
  def say(_),                              do: {:error, "not a positive integer"}

  defp do_say(n) when rem(n, 15) == 0, do: "FizzBuzz"
  defp do_say(n) when rem(n, 5)  == 0, do: "Buzz"
  defp do_say(n) when rem(n, 3)  == 0, do: "Fizz"
  defp do_say(n),                      do: n
end

defmodule FizzBuzz.Client do
  def call_fizzbuzz(n) do
    make_pair_with_fizzbuzz(n) |> format_result()
  end

  defp make_pair_with_fizzbuzz(n) do
    {n, FizzBuzz.say(n)}
  end
  
  defp format_result({n, {:ok,    value}}),  do: "FizzBuzz(#{n}) = #{value}"
  defp format_result({n, {:error, reason}}), do: "FizzBuzz(#{inspect(n)}) failed: #{reason}"
end

In [ ]:
1..30
|> Enum.map(&FizzBuzz.Client.call_fizzbuzz/1)
|> Enum.each(&IO.puts/1)


[-1, 0, 1.0, "a", [1, 2, 3], %{x: 0}]
|> Enum.map(&FizzBuzz.Client.call_fizzbuzz/1)
|> Enum.each(&IO.puts/1)